# Final Project for the lecture "Computer Vision and 3D reconstruction"
Winter term 2022 /23

Load dependencies


In [1]:
from utils.CreateTestTrainValSplit import spliter

Set up the training, validation and test sets

In [2]:
spliter(source_X="./semantic_drone_dataset/original_images/",source_Y="./semantic_drone_dataset/label_images_semantic/",val_size=50,test_size=50)

Number of instaces: 400
instance 0:	 training set
instance 1:	 training set
instance 2:	 validation set
instance 3:	 training set
instance 4:	 training set
instance 5:	 training set
instance 6:	 training set
instance 7:	 training set
instance 8:	 test set
instance 9:	 test set
instance 10:	 test set
instance 11:	 training set
instance 12:	 training set
instance 13:	 training set
instance 14:	 training set
instance 15:	 training set
instance 16:	 training set
instance 17:	 training set
instance 18:	 training set
instance 19:	 validation set
instance 20:	 training set
instance 21:	 training set
instance 22:	 training set
instance 23:	 training set
instance 24:	 training set
instance 25:	 training set
instance 26:	 training set
instance 27:	 training set
instance 28:	 training set
instance 29:	 training set
instance 30:	 training set
instance 31:	 training set
instance 32:	 training set
instance 33:	 training set
instance 34:	 test set
instance 35:	 training set
instance 36:	 training set